In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

# 1. 패키지
- python-dotenv, langchain(1.2.0), langchain-openai, langchain-pinecone, pandas, langchain-community, docx2txt, langchain-text-splitters, langchain-ollama

# 2. 환경설정(env, 시스템파라미터변수)

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

# 3. 문서를 chunk로 분할하기

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./data/소득세법with_md.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)
# documents = loader.load_and_split(text_splitter)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 chunk 생성")

총 194개 chunk 생성


In [7]:
documents[0].page_content

'소득세법\n\n소득세법\n\n[시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자\n\

# 4. metadata 추가하기
- chunk 내용의 카테고리, chunk 내용의 title, 조항구분

In [13]:
import re
def remove_special_chars(text:str) -> str:
    "특수문자 및 \n 제거(:는 그대로)"
    # \n 제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, : 말고는 전무 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,\.:]', "", text)
    # 불필요한 중복공백 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned
# test
# content = "**소득세 납세 의무**: \n\n이 법 또는 「법인세법」에 따라 소득에 대한 소득세를 부과 <2009. 12. 31>"
# print(remove_special_chars(content))

소득세 납세 의무: 이 법 또는 법인세법에 따라 소득에 대한 소득세를 부과 2009. 12. 31


## 4.1. 제목

In [14]:
# 제목을 추출하는 함수(exaone3.5) : cmd에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extract_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
        num_predict = 30 # 최대 토큰 30개까지만 출력
    )
    content = remove_special_chars(content)
    prompt = f"""다음 소득세법 조문의 핵심 제목을 30토큰 이내로 간단히 추출해 주세요. 중간에 말이 끊기지 않고 온전히 말이 되도록 합니다.
    예: 소득세 납세의미 범위: 공동사업자별 소득과세, 상속과세, 증여자
    조문: {content}"""
    ai_message = title_extract_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)
# test
# content = """제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자\n\n3. 내국법인\n\n4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 이하 같다)\n\n5. 그 밖에 이 법에서 정하는 원천징수의무자\n\n③ 「국세기본법」 제13조제1항에 따른 법인 아닌 단체 중 같은 조 제4항에 따른 법인으로 보는 단체(이하 “법인으로 보는 단체”라 한다) 외의 법인 아닌 단체는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 경우에는 1거주자로, 그 밖의 경우에는 1비거주자로 보아 이 법을 적용한다. 다만, 다음 각 호의 어느 하나에 해당하는 경우에는 소득구분에 따라 해당 단체의 각 구성원별로 이 법 또는 「법인세법」에 따라 소득에 대한 소득세 또는 법인세[해당 구성원이 「법인세법」에 따른 법인(법인으로 보는 단체를 포함한다)인 경우로 한정한다. 이하 이 조에서 같다]를 납부할 의무를 진다.<개정 2010. 12. 27., 2013. 1. 1., 2018. 12. 31.>"""
# print(extract_title_with_llm(content))

소득세 납세 의무: 거주자 및 국내원천소득 비거주자에게 부과


## 4.2. 카테고리

In [20]:
# 카테고리 반환하는 함수 (1)
def categorize_content(content):
    "내용 카테고리 분류"
    categories = {
        '납세의무': ['납세의무', '거주자', '비거주자', '원천징수', '공동사업자', '상속인', '증여자', '신탁재산'],
        '세율계산': ['세율', '과세표준', '산출세액', '결정세액', '종합소득', '퇴직소득'],
        '근로소득': ['근로소득', '총급여', '급여', '연봉', '임금', '퇴직소득'],
        '사업소득': ['사업소득', '공동사업', '주택임대소득', '부업소득'],
        '이자배당': ['이자소득', '배당소득', '예금이자', '채권', '의제배당'],
        '양도소득': ['양도소득', '자산양도', '부동산양도', '주식양도'],
        '연금소득': ['연금소득', '공적연금', '사적연금', '연금보험'],
        '기타소득': ['기타소득', '상금', '보상금', '발명보상금', '종교인소득'],
        '공제감면': ['공제', '소득공제', '세액공제', '기본공제', '감면'],
        '비과세': ['비과세', '면제', '복무급여', '실업급여', '출산휴가급여', '장학금'],
        '신고납부': ['신고', '납부', '납세지', '신고기한', '원천징수'],
        '과세기간': ['과세기간', '과세연도', '사업연도'],
        '과세소득구분': ['종합소득', '퇴직소득', '양도소득', '금융투자소득'],
    }
    result = [] # 카테고리 추가
    for category, keywords in categories.items():
        if any([keyword in content for keyword in keywords]):
            result.append(category)
    if not result:
        result.append("기타")
    return result
categorize_content(content)

['납세의무', '사업소득', '신고납부']

- 개선된 카테고리 분류방법 : 키워드 중요도에 따라 가중치 부여, 가중치 높은 순으로 카테고리 반환

In [21]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },
        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },
        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },
        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },
        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },
        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },
        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },
        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },
        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },
        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },
        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },
        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },
        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },
        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [22]:
# test 조문 content
content = """제17조(배당소득) ① 배당소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2012. 1. 1., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31., 2024. 12. 31.>
1. 내국법인으로부터 받는 이익이나 잉여금의 배당 또는 분배금
2. 법인으로 보는 단체로부터 받는 배당금 또는 분배금
2의2. 「법인세법」 제5조제2항에 따라 내국법인으로 보는 신탁재산(이하 “법인과세 신탁재산”이라 한다)으로부터 받는 배당금 또는 분배금
3. 의제배당(擬制配當)
4. 「법인세법」에 따라 배당으로 처분된 금액
5. 국내 또는 국외에서 받는 대통령령으로 정하는 집합투자기구로부터의 이익
5의2. 국내 또는 국외에서 받는 대통령령으로 정하는 파생결합증권 또는 파생결합사채로부터의 이익
5의3. 금전이 아닌 재산의 신탁계약에 의한 수익권이 표시된 수익증권으로서 대통령령으로 정하는 수익증권으로부터의 이익
5의4. 「자본시장과 금융투자업에 관한 법률」 제4조제6항에 따른 투자계약증권으로서 대통령령으로 정하는 투자계약증권으로부터의 이익
"""

In [33]:
# 카테고리 반환하는 함수 (2)
def categorize_content(content, top_k=None):
    """
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters:
    - content: 분류할 텍스트 내용
    - top_k: 상위 몇 개까지 카테고리 반환할지(None이면 관련된 모든 카테고리명 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)
    """
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            if keyword in content:
                score += weight
        if score:
            category_scores[category] = score
    # value로 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1], reverse=True) # 내림차순
    all_categories = [category[0] for category in sorted_categories]
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
    # top_k None이 아니면 상위 top_k개만, None이면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories
categorize_content(content, 2)

['이자배당', '과세기간']

In [34]:
categorize_content("연봉 5천만 원인 회사원의 소득세는 얼마예요?")

['세율계산', '근로소득']

In [36]:
categorize_content(documents[46].page_content)

['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간']

## 4.3. 조항

In [41]:
# 해당 조항 추출하기
import re
def get_article(content):
    "내용 중 관련 조항 추출"
    article_match = re.findall(r'제(\d+)조', content)
    article = list(set(article_match)) # 중복 제거
    article.sort(key=lambda x: int(x))
    if article:
        return "「" + ",".join([f"{a}조" for a in article]) + "」"
get_article("제267조, 제2조, 제99조, 제267조, 제99조")

'「2조,99조,267조」'

In [45]:
%%time
# metadata: source, title, category, article
enhanced_chunks = []
for i, chunk in enumerate(documents):
    if i%20 == 0:
        print(f"진행 중: {i/len(documents)*100:.2f}% 진행")
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    
    # exaone으로 title 추출
    # metadata['title'] = extract_title_with_llm(content)
    # category 추출
    metadata['category'] = categorize_content(content)
    # id 생성
    metadata['chunk_id'] = f"chunk_{i:03d}"
    # 조항(article) 추출
    article = get_article(content)
    if article:
        metadata['article'] = article
    enhanced_chunks.append(type(chunk)(page_content=content, metadata=metadata)) #document 타입 맞추기
print("확장된 chunk 처리 완료")

진행 중: 0.00% 진행
진행 중: 10.31% 진행
진행 중: 20.62% 진행
진행 중: 30.93% 진행
진행 중: 41.24% 진행
진행 중: 51.55% 진행
진행 중: 61.86% 진행
진행 중: 72.16% 진행
진행 중: 82.47% 진행
진행 중: 92.78% 진행
확장된 chunk 처리 완료
CPU times: total: 15.6 ms
Wall time: 24.8 ms


# 5. Embedding Model 설정

In [46]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model = OPENAI_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY
)

# 6. Pinecone Index 생성 및 vector store(DB) 저장

In [48]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# pinecone client 객체
pc = Pinecone(api_key=PINECONE_API_KEY)
# print("pinecone index 조회:", pc.list_indexes().names())

# INDEX 생성 여부 확인 및 생성
# if PINECONE_INDEX_NAME not in pc.list_indexes().names():
if not pc.has_index(PINECONE_INDEX_NAME):
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DEMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION,
                           cloud=PINECONE_INDEX_CLOUD)
    )
    print('INDEX 생성 완료')
else:
    print(f'INDEX {PINECONE_INDEX_NAME}이 이미 존재합니다.')

INDEX 생성 완료


In [51]:
%%time
# Pinecone vector store에 upload
vector_database = PineconeVectorStore.from_documents(
    documents=enhanced_chunks,
    embedding=embedding,
    index_name=PINECONE_INDEX_NAME
)
print("Vector DB 저장 완료")

Vector DB 저장 완료
CPU times: total: 4.44 s
Wall time: 15.7 s


# 7. 유사도 검색(metadata 활용)

In [52]:
query = "연봉 5천만 원인 직장인의 소득세는 얼마예요?"
categorize_content(query)

['근로소득', '세율계산']

In [ ]:
# Retriever 생성
retriever = vector_database.as_retriever(
    search_kwargs={
        "k":3,
        "filter":{"category":{"$in":categorize_content(query)}}
    }
)
docs = retriever.invoke(query)
print("관련 문서")

In [55]:
for i, doc in enumerate(docs):
    category = doc.metadata['category']
    article = doc.metadata.get('article', '조항 없음') #article 빈 chunk는 get 안 하면 에러 생길 수 있어서
    content = doc.page_content[:20]
    print(f"{i+1}.{article} {category} - {content}")

관련 문서
1.「20조,21조,25조,32조,164조」 ['기타소득', '연금소득', '근로소득'] - 2) 대학의 교직원 또는 대학과 고용
2.「14조,17조,55조,56조」 ['세율계산', '이자배당', '공제감면', '과세방식', '납세의무', '사업소득', '과세기간'] - 제55조(세율) ①거주자의 종합소득에
3.「6조,9조,10조,11조,12조,99조」 ['납세의무', '사업소득', '세율계산', '근로소득', '비과세', '신고납부', '과세기간'] - [전문개정 2009. 12. 31.]


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough